In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from admm_approximate_solver import Approximate_ReLU_ADMM_Solver
from relu_utils import squared_loss, classifcation_accuracy

In [2]:
# generate toy data trying to fit noisy data to cosine
data = pd.read_csv("test_data/wineQuality-red.csv", delimiter=";")
X = np.array(data)[:,:11]
y = np.array(data.quality)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_train, d = X_train.shape
n_test = X_test.shape[0]

print(f"n train = {n_train}")
print(f"n test = {n_test}")

m = 10

n train = 1279
n test = 320


In [3]:
# TODO: solve simple pytorch problem

MLP = nn.Sequential(nn.Linear(d, m),
                    nn.ReLU(),
                    nn.Linear(m, np.max(y)+1))

print(MLP)

Sequential(
  (0): Linear(in_features=11, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=9, bias=True)
)


Solver doesn't converge well... Things needing fixing right now: 

- How to choose m, P_S?
- How to choose parameters $\rho$, step=$\gamma_{\alpha}$, $\beta$? 
- Converting from optimal weights $v, w$ to $u, \alpha$. Currently loss during training is consistently high but gets lower when calling .predict

In [4]:
# solve cvx problem

solver = Approximate_ReLU_ADMM_Solver(m=10,
                                      P_S=10,
                                      rho=0.0001,
                                      step=0.0001,
                                      beta=0.0001,
                                      bias=True,
                                      loss_func=squared_loss,
                                      acc_func=classifcation_accuracy)

solver.optimize(X_train, y_train, max_iter=100, verbose=True)


print("CONVEX PROBLEM WEIGHTS:")
y_hat_train = solver.predict(X_train, weights="C-ReLU")
y_hat_test = solver.predict(X_test, weights="C-ReLU")
print(f"Train loss: {squared_loss(y_hat_train, y_train)}")
print(f"Train accuracy: {classifcation_accuracy(y_hat_train, y_train)}")
print(f"Test loss: {squared_loss(y_hat_test, y_test)}")
print(f"Test accuracy: {classifcation_accuracy(y_hat_test, y_test)}")

print("NONCONVEX PROBLEM WEIGHTS:")
y_hat_train = solver.predict(X_train, weights="NC-ReLU")
y_hat_test = solver.predict(X_test, weights="NC-ReLU")
print(f"Train loss: {squared_loss(y_hat_train, y_train)}")
print(f"Train accuracy: {classifcation_accuracy(y_hat_train, y_train)}")
print(f"Test loss: {squared_loss(y_hat_test, y_test)}")
print(f"Test accuracy: {classifcation_accuracy(y_hat_test, y_test)}")

iter = 0, loss = 20642.500000000004, acc = 0.0
iter = 1, loss = 241.66529567373126, acc = 0.6317435496481626
iter = 2, loss = 241.4982586121236, acc = 0.6309616888193902
iter = 3, loss = 241.36039044967305, acc = 0.6309616888193902
iter = 4, loss = 241.2805228628327, acc = 0.6309616888193902
iter = 5, loss = 241.23780168681904, acc = 0.6293979671618452
iter = 6, loss = 241.2144015295555, acc = 0.6301798279906177
iter = 7, loss = 241.20166820306642, acc = 0.6325254104769351
iter = 8, loss = 241.1948170469304, acc = 0.6333072713057076
iter = 9, loss = 241.19128340761014, acc = 0.6333072713057076
iter = 10, loss = 241.1897577444754, acc = 0.63408913213448
iter = 11, loss = 241.1895061919421, acc = 0.63408913213448
iter = 12, loss = 241.18994266970515, acc = 0.6325254104769351
iter = 13, loss = 241.19079382872096, acc = 0.6325254104769351
iter = 14, loss = 241.19208672846625, acc = 0.6325254104769351
iter = 15, loss = 241.19410465684916, acc = 0.6325254104769351
iter = 16, loss = 241.19654

/Users/zachary/Desktop/Stanford/_Spr2023/EE 364B/Project/baADMM/admm_approximate_solver.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  v[inds] = np.maximum(1 - self.beta/(self.rho * LA.norm(u[inds] + lam[inds])), 0) * (u[inds] + lam[inds])


iter = 18, loss = 241.20215720335898, acc = 0.6325254104769351
iter = 19, loss = 241.20522733934948, acc = 0.6333072713057076
iter = 20, loss = 241.20846536022057, acc = 0.6325254104769351
iter = 21, loss = 241.2118930057887, acc = 0.6325254104769351
iter = 22, loss = 241.2155004269168, acc = 0.6325254104769351
iter = 23, loss = 241.21944978968895, acc = 0.6325254104769351
iter = 24, loss = 241.22365507697816, acc = 0.6325254104769351
iter = 25, loss = 241.22813634227836, acc = 0.6325254104769351
iter = 26, loss = 241.2327291244362, acc = 0.6325254104769351
iter = 27, loss = 241.2373226776444, acc = 0.6325254104769351
iter = 28, loss = 241.24210967529524, acc = 0.6325254104769351
iter = 29, loss = 241.24693127772593, acc = 0.6325254104769351
iter = 30, loss = 241.251828972139, acc = 0.6325254104769351
iter = 31, loss = 241.25680100260658, acc = 0.6317435496481626
iter = 32, loss = 241.26190161092327, acc = 0.6317435496481626
iter = 33, loss = 241.26698686778184, acc = 0.631743549648162

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 320 is different from 1279)